In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics

In [2]:
burns=pd.read_csv("data2/burns-trauma.csv")

In [3]:
apachepredvar = pd.read_csv('apachePredVar.csv.gz')

In [4]:
burns_merged = pd.merge(apachepredvar[['patientunitstayid','pao2','fio2']], burns, how='right', on='patientunitstayid')

In [5]:
burns_merged['sofa_ratio'] = 0

In [6]:
for i in range(len(burns_merged)):
    if (burns_merged.iloc[i]['pao2'] != -1) & (burns_merged.iloc[i]['fio2'] != -1):
        if ((burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 <= 399) & ((burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 >= 300):
            burns_merged['sofa_ratio'].iloc[i] = 1
        if ((burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 <= 299) & ((burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 >= 200):
            burns_merged['sofa_ratio'].iloc[i] = 2
        if ((burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 <= 199) & (burns_merged.iloc[i]['ventday1'] != 1):
            burns_merged['sofa_ratio'].iloc[i] = 2
        if ((burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 >= 100) & ((burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 <= 199) & (burns_merged.iloc[i]['ventday1'] == 1):
            burns_merged['sofa_ratio'].iloc[i] = 3
        if ((burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 < 100) & (burns_merged.iloc[i]['ventday1'] == 1):
            burns_merged['sofa_ratio'].iloc[i] = 4

C:\Users\nsafa\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [7]:
def platelets(item):
    temp = 0
    if (item >= 100) & (item <= 149):
        temp += 1
    elif (50 <= item) & (item <= 99):
        temp += 2
    elif (20 <= item) & (item <= 49):
        temp += 3
    elif (20 > item):
        temp += 4
    return temp

In [8]:
burns_merged['sofa_platelets'] = burns_merged['platelets x 1000'].apply(platelets)

In [9]:
burns_merged['GCS'] = burns_merged['eyes'] + burns_merged['motor'] + burns_merged['verbal']

In [10]:
def GCS(item):
    temp = 0
    if (item >= 13) & (item <= 14):
        temp += 1
    elif (10 <= item) & (item <= 12):
        temp += 2
    elif (6 <= item) & (item <= 9):
        temp += 3
    elif (6 > item):
        temp += 4
    return temp

In [11]:
burns_merged['sofa_GCS'] = burns_merged['GCS'].apply(GCS)

In [12]:
apacheapsdvar = pd.read_csv('ApacheApsVar.csv.gz')

In [13]:
burns_merged_1 = pd.merge(apacheapsdvar[['patientunitstayid','bilirubin']], burns_merged, how='right', on='patientunitstayid')

In [14]:
def bilirubin(item):
    temp = 0
    if (item >= 1.2) & (item <= 1.9):
        temp += 1
    elif (2 <= item) & (item <= 5.9):
        temp += 2
    elif (6 <= item) & (item <= 11.9):
        temp += 3
    elif (12 <= item):
        temp += 4
    return temp

In [15]:
burns_merged_1['sofa_bilirubin'] = burns_merged_1['bilirubin'].apply(bilirubin)

In [16]:
def meanbp(item):
    temp = 0
    if (item < 70):
        temp += 1
    return temp

In [17]:
burns_merged_1['sofa_meanbp'] = burns_merged_1['meanbp'].apply(meanbp)

In [18]:
burns_merged_1['creatinine'].mean()

1.1235210522389558

In [19]:
apacheapsdvar = pd.read_csv('ApacheApsVar.csv.gz')

In [20]:
burns_merged_2 = pd.merge(apacheapsdvar[['patientunitstayid','urine']], burns_merged_1, how='right', on='patientunitstayid')

In [21]:
burns_merged_2['sofa_creatinine_urine'] = 0

In [22]:
for i in range(len(burns_merged_2)):
    if (burns_merged_2['creatinine'].iloc[i] >= 1.2) & (burns_merged_2['creatinine'].iloc[i] <= 1.9):
        burns_merged_2['sofa_creatinine_urine'].iloc[i] = 1
    elif (burns_merged_2['creatinine'].iloc[i] >= 2) & (burns_merged_2['creatinine'].iloc[i] <= 3.4):
        burns_merged_2['sofa_creatinine_urine'].iloc[i] = 2
    elif ((burns_merged_2['creatinine'].iloc[i] >= 3.5) & (burns_merged_2['creatinine'].iloc[i] <= 4.9)) | ((burns_merged_2['urine'].iloc[i] < 500) & (burns_merged_2['urine'].iloc[i] != -1)):
        burns_merged_2['sofa_creatinine_urine'].iloc[i] = 3
    elif (burns_merged_2['creatinine'].iloc[i] >= 5) | ((burns_merged_2['urine'].iloc[i] < 200) & (burns_merged_2['urine'].iloc[i] != -1)):
        burns_merged_2['sofa_creatinine_urine'].iloc[i] = 4

In [23]:
burns_merged_2['sofa'] = burns_merged_2[['sofa_ratio', 'sofa_platelets', 'sofa_GCS', 'sofa_bilirubin',
       'sofa_meanbp', 'sofa_creatinine_urine']].sum(axis=1)

In [24]:
def sofa_mortality(item):
    if (0 <= item) & (item <=1):
        return 0
    elif (2 <= item) & (item <=3):
        return 1.5
    elif (4 <= item) & (item <=5):
        return 6.7
    elif (6 <= item) & (item <=7):
        return 18.2
    elif (8 <= item) & (item <=9):
        return 26.3
    elif (10 <= item) & (item <=11):
        return 45.8
    elif (12 <= item) & (item <=14):
        return 80
    elif (14 < item):
        return 89.7

In [25]:
burns_merged_2['sofa_mortality'] = burns_merged_2['sofa'].apply(sofa_mortality)

In [26]:
burns_merged_2

,patientunitstayid,urine,bilirubin,pao2,fio2,gender,age,ethnicity,apacheadmissiondx,admissionheight,...,RDW,sofa_ratio,sofa_platelets,GCS,sofa_GCS,sofa_bilirubin,sofa_meanbp,sofa_creatinine_urine,sofa,sofa_mortality
0,143103,-1.0000,0.3,346.0,100.0,Female,46,Asian,Head only trauma,167.6,...,13.50,1,0,6.0,3,0,0,0,4,6.7
1,143258,-1.0000,1.0,-1.0,-1.0,Male,90,Caucasian,"Rhythm disturbance (atrial, supraventricular)",180.3,...,12.10,0,0,15.0,0,0,0,1,1,0.0
2,143274,-1.0000,-1.0,-1.0,-1.0,Male,22,Caucasian,Head/extremity trauma,195.6,...,12.70,0,0,13.0,1,0,0,1,2,1.5
3,145180,-1.0000,-1.0,-1.0,-1.0,Female,77,-,"Extremity only trauma, surgery for",167.6,...,13.70,0,0,11.0,2,0,0,0,2,1.5
4,146784,-1.0000,-1.0,80.0,40.0,Male,50,Caucasian,Spinal/face trauma,177.8,...,12.90,2,0,14.0,1,0,1,0,4,6.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4910,3352198,925.1712,-1.0,-1.0,-1.0,Male,61,Other/Unknown,Chest/face trauma,188.0,...,13.60,0,0,15.0,0,0,0,0,0,0.0
4911,3352519,1419.2064,-1.0,-1.0,-1.0,Male,46,Caucasian,Head/multiple trauma,172.7,...,13.30,0,0,14.0,1,0,0,0,1,0.0
4912,3352711,2121.2064,-1.0,112.0,100.0,Male,53,Caucasian,"Sepsis, pulmonary",172.7,...,13.60,3,0,14.0,1,0,1,0,5,6.7
4913,3353007,2178.5760,-1.0,-1.0,-1.0,Female,74,Caucasian,Chest/extremity trauma,157.5,...,12.75,0,0,15.0,0,0,0,0,0,0.0


In [27]:
burns_merged_2['actualicumortality_label'] = burns_merged_2['actualicumortality'].apply(lambda x: 1 if x =='EXPIRED' else 0)

In [28]:
fpr, tpr, thresholds = metrics.roc_curve(burns_merged_2['actualicumortality_label'].values, burns_merged_2['sofa_mortality'].values, pos_label=1)

In [29]:
metrics.auc(fpr, tpr)

0.8362808213414121